In [34]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from plotly.subplots import make_subplots


### Exploratory analysis

In [35]:
priority_wells = ['AW5D', 'AW5O', 'AW5S','AW6D', 'AW6O', 'AW6S','BW3D', 'LRS69D', 'LRS69O', 'LRS69S','LRS69DR', 'LRS69OR', 'LRS69SR' ,'LRS70D', 'LRS70S']

#### Caliper

In [36]:
# path = r'notebooks\sandbox\concatenate_caliper_all.csv'
caliper= pd.read_csv('concatenate_caliper_all.csv')
caliper.sample(5)

,Unnamed: 0,Depth_ft,Caliper_in,source_file,Diameter_auger_in,calibrated_in,calibrated_cm,Well_Diameter_Deviation_Percentage,Depth [m]
14572,14572,40.68130,5.73281,BW4D_caliper_20210910.LAS,6,7.126487,18.101277,18.774783,-12.399660
127,127,72.19330,5.11264,AW1D_caliper_20210910.LAS,6,6.478273,16.454814,7.971221,-22.004518
7841,7841,6.68435,2.83437,AW7O_caliper_20210910.LAS,4,4.096981,10.406332,2.424530,-2.037390
4994,4994,81.86380,5.02763,AW6D_caliper_20210910.LAS,6,6.389419,16.229125,6.490319,-24.952086
8862,8862,92.53280,5.90282,BW10D_caliper_20211020.LAS,6,7.304185,18.552629,21.736412,-28.203997


In [37]:
# Retain only the wells defined as priority for the current analysis
# Extract well name from 'source_file' by taking characters before the first underscore
caliper['Well'] = caliper['source_file'].str.extract(r'^([^_]+)')
caliper_priority = caliper[caliper['Well'].isin(priority_wells)].copy()
caliper_priority.sample(5)


,Unnamed: 0,Depth_ft,Caliper_in,source_file,Diameter_auger_in,calibrated_in,calibrated_cm,Well_Diameter_Deviation_Percentage,Depth [m],Well
5630,5630,18.44880,5.80328,AW6D_caliper_20210910.LAS,6,7.200144,18.288365,20.002393,-5.623194,AW6D
23226,23226,18.44880,8.23925,LRS70D_caliper_20210910.LAS,8,9.746267,24.755518,21.828334,-5.623194,LRS70D
4548,4548,27.92240,2.78683,AW5O_caliper_20210910.LAS,4,4.047291,10.280120,1.182287,-8.510748,AW5O
22939,22939,47.06530,7.52121,LRS70D_caliper_20210910.LAS,8,8.995757,22.849224,12.446967,-14.345503,LRS70D
6674,6674,4.38976,4.58865,AW6S_caliper_20210910.LAS,6,5.930589,15.063695,-1.156854,-1.337999,AW6S


In [38]:
caliper.columns

Index(['Unnamed: 0', 'Depth_ft', 'Caliper_in', 'source_file',
       'Diameter_auger_in', 'calibrated_in', 'calibrated_cm',
       'Well_Diameter_Deviation_Percentage', 'Depth [m]', 'Well'],
      dtype='object')

In [39]:
print(f"Row count for each well:\n{caliper_priority['Well'].value_counts()}\n\n"
      f"Total priority wells defined: {len(priority_wells)}\n"
      f"Total priority wells after filtering: {caliper_priority['Well'].nunique()}")


Row count for each well:
Well
LRS70D    1160
BW3D      1002
AW5D       900
AW6D       898
LRS69D     865
AW6O       753
AW5O       343
LRS69S     245
LRS70S     203
AW6S       147
AW5S        95
Name: count, dtype: int64

Total priority wells defined: 15
Total priority wells after filtering: 11


In [40]:
# It seems LRS69O is missing from the caliper data — let's check which LRS69 wells are actually present
caliper[caliper['source_file'].str.startswith('LRS70')]['Well'].unique()


array(['LRS70D', 'LRS70S'], dtype=object)

#### Conductivity

In [41]:
## Standarizing column names across csvs
# Set to collect column names
column_names = set()

# Path to CSV folder
csv_folder = Path("../../data/raw")
csv_paths = list(csv_folder.glob("*.csv"))

# Loop through only priority well files
for path in csv_paths:
    well_id = path.stem.split("_")[0]  # Get well ID from filename
    if well_id in priority_wells:
        try:
            df = pd.read_csv(path, nrows=1)  # Read only the header
            print(f"{path.name}: {len(df.columns)} columns")
            column_names.update(df.columns)
        except Exception as e:
            print(f"{path.name}: ❌ Error - {e}")
print(column_names)

AW5D_YSI_20250225.csv: 20 columns
AW5O_YSI_20250225.csv: 20 columns
AW5S_YSI_20250225.csv: 23 columns
AW6D_YSI_20250226.csv: 20 columns
AW6O_YSI_20250226.csv: 20 columns
AW6S_YSI_20250226.csv: 20 columns
BW3D_YSI_20250222.csv: 23 columns
LRS69DR_YSI_20250222R.csv: 21 columns
LRS69D_YSI_20250222.csv: 23 columns
LRS69OR_YSI_20250222R.csv: 21 columns
LRS69O_YSI_20250222.csv: 23 columns
LRS69SR_YSI_20250222R.csv: 27 columns
LRS69S_YSI_20250222.csv: 23 columns
LRS70D_YSI_20250317.csv: 21 columns
LRS70S_YSI_20250317.csv: 21 columns
{'Pressure psi a', 'Vertical Position m', 'Depth m.1', 'nLF_Cond_muS/cm', 'Unnamed: 20', 'ORP mV', 'Density Sigma', 'SpCond µS/cm', 'Unnamed: 23', 'Cond µS/cm', 'Rhodamine WT RFU', 'ODO % CB', 'TDS mg/L', 'Sal psu', 'Battery V', 'Rhodamine WT ug/L', 'Temp_Celcius', 'ODO mg/L', 'Unnamed: 25', 'Time (Fract. Sec)', 'pH mV', 'Cond_muS/cm', 'Unnamed: 21', 'SpCond_muS/cm.1', 'ODO % sat', 'SpCond_muS/cm', 'Cable Pwr V', 'pH', 'Density-T Sigma-T', 'Depth m', 'Date (MM/DD/

In [42]:
rename_dict = {
    # Conductividad específica
    'SpCond_muS/cm': 'SpCond µS/cm',
    'nLF_Cond_muS/cm': 'nLF Cond µS/cm',
    'Cond_muS/cm': 'Cond µS/cm',
    #'Corrected sp Cond [µS/cm]': 'SpCond_muS/cm',
   
    # Temperatura
    'Temp °C': 'Temperature_C',
    'Temp_Celcius': 'Temperature_C',

    # Profundidad
    #'Depth from GL (m)': 'Depth_m',
    #'Corrected Depth': 'Depth_m',

    # Tiempo
    'Time (HH:MM:SS)': 'Time (HH:mm:ss)',
    'Date (MM/DD/YYYY)': 'Date',


}


In [43]:
dfs=[]
for path in csv_paths:
    well_id = path.stem.split("_")[0]
    if well_id in priority_wells:
        df = pd.read_csv(path)
        df["Well"] = well_id
        df.rename(columns=rename_dict, inplace=True)
        df.drop(columns=[col for col in df.columns if col.startswith('Unnamed')], inplace=True)
        dfs.append(df)

# Combine all into a single DataFrame
if dfs:
    conductivity = pd.concat(dfs, ignore_index=True)
    
else:
    print("⚠️ No matching CSV files from priority wells were found or loaded.")

conductivity.sample(6)

,Date,Time (HH:mm:ss),Time (Fract. Sec),Site Name,Cond µS/cm,Depth m,ODO % sat,ODO mg/L,ORP mV,Pressure psi a,...,Resistivity ohms-cm,Well,Depth m.1,Rhodamine WT RFU,Rhodamine WT ug/L,Density Sigma,Density-T Sigma-T,nLF Cond µS/cm,ODO % CB,SpCond_muS/cm.1
14566,25/02/2025,09:40:50,0.0,Default Site,648.8,1.664,22.3,1.85,225.6,2.361,...,1541.0,AW5O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22258,26/02/2025,10:44:28,0.0,Default Site,537.2,2.954,19.2,1.58,92.1,4.190,...,1862.0,AW6D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7096,25/02/2025,13:00:44,0.0,Default Site,7551.5,14.588,0.0,0.00,126.6,20.751,...,132.0,AW5D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41505,22/2/2025,16:42:55,0.0,Default Site,501.0,3.017,28.1,2.27,226.8,4.277,...,NaN,BW3D,NaN,-2.21,-22.05,-3.1,-3.1,NaN,NaN,NaN
13354,25/02/2025,14:45:02,0.0,Default Site,49518.5,24.246,0.1,0.01,-262.9,35.225,...,20.0,AW5D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105841,22/02/2025,14:57:57,0.5,Default Site,54744.7,24.030,1.8,0.12,-231.4,34.997,...,NaN,LRS69DR,NaN,NaN,NaN,NaN,NaN,54296.0,1.8,NaN


##### Correccion Zona Vadosa

In [44]:
# --- 1. Load vadose zone data ---
# Path
vadose_path = 'vz_aug2023.csv'
vadose = pd.read_csv(vadose_path)

# Normalize column names (lowercase, no extra spaces)
vadose.columns = vadose.columns.str.strip().str.lower()

# Map possible variants to standard column names
col_map = {}
for col in vadose.columns:
    if 'well' in col and not col_map.get('well'):
        col_map[col] = 'Well'
    elif 'vado' in col or 'vz' in col or 'zone' in col:
        col_map[col] = 'vz_thickness_m'

# Rename only if required columns found
if set(col_map.values()) >= {'Well', 'vz_thickness_m'}:
    vadose = vadose.rename(columns=col_map)
else:
    raise ValueError("❌ Required columns for merge not found in vadose zone file (need 'Well' and 'vz_thickness_m')")


#  --- 2. Apply depth correction to conductivity data ---

# Make sure 'Well' is a clean string in both DataFrames
conductivity['Well'] = conductivity['Well'].astype(str).str.strip()

# Merge VZ values to conductivity data based on well name
conductivity_corr = conductivity.merge(
    vadose[['Well', 'vz_thickness_m']],
    on='Well',
    how='left'
)

# Compute depth from ground level (GL) only if VZ is available
conductivity_corr['Depth_from_GL'] = conductivity_corr['Vertical Position m'] + conductivity_corr['vz_thickness_m']

# Optional: warn if some wells are missing VZ data
missing_toc = conductivity_corr['vz_thickness_m'].isna().sum()
if missing_toc > 0:
    missing_wells = conductivity_corr.loc[conductivity_corr['vz_thickness_m'].isna(), 'Well'].unique()
    print("Wells without VZ data:")
    for well in missing_wells:
        print(f"  - {well}")


Wells without VZ data:
  - LRS69DR
  - LRS69OR
  - LRS69SR


In [45]:
conductivity_corr[['Well','Vertical Position m', 'vz_thickness_m', 'Depth_from_GL']]

,Well,Vertical Position m,vz_thickness_m,Depth_from_GL
0,AW5D,0.039,0.270,0.309
1,AW5D,0.041,0.270,0.311
2,AW5D,0.041,0.270,0.311
3,AW5D,0.041,0.270,0.311
4,AW5D,0.040,0.270,0.310
...,...,...,...,...
152687,LRS70S,1.147,0.635,1.782
152688,LRS70S,1.149,0.635,1.784
152689,LRS70S,1.150,0.635,1.785
152690,LRS70S,1.149,0.635,1.784


### Comparison graph

In [46]:
# ---------------------
# 🔁 Standardize Depth
# ---------------------

# Caliper and Conductivity dataframes use different depth conventions
# Caliper: "Depth [m]" (negative values), Conductivity: "Vertical Position m" (positive values)
# We'll standardize both to a common column: Depth_m, with positive depth (downward)

caliper["Depth_m"] = caliper["Depth [m]"].abs()
# conductivity["Depth_m"] = conductivity["Vertical Position m"]
conductivity_corr["Depth_m"] = conductivity_corr["Depth_from_GL"]


# Create the base figure with shared Y axis
fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=True,
    horizontal_spacing=0.05,
    subplot_titles=("Conductivity (µS/cm)", "Caliper")
)

# Store visibility masks and buttons
visibility = []
buttons = []

# Loop through wells and add their data as hidden traces by default
for i, well in enumerate(sorted(set(caliper["Well"]).intersection(conductivity_corr["Well"]))):
    cal = caliper[caliper["Well"] == well]
    cond = conductivity_corr[conductivity_corr["Well"] == well]

    # Add conductivity trace (col 1)
    fig.add_trace(
        go.Scatter(
            x=cond["SpCond µS/cm"],
            y=cond["Depth_m"],
            mode="markers",
            name=f"{well} - Conductivity",
            marker=dict(color="red", size=4),
            visible=(i == 0)  # Only the first well is visible initially
        ),
        row=1, col=1
    )

    # Add caliper trace (col 2)
    fig.add_trace(
        go.Scatter(
            x=cal["calibrated_cm"],
            y=cal["Depth_m"],
            mode="markers",
            name=f"{well} - Caliper",
            marker=dict(color="blue", size=4),
            visible=(i == 0)
        ),
        row=1, col=2
    )

    # Build visibility array for this well (2 traces per well)
    vis = [False] * (2 * i) + [True, True] + [False] * (2 * (len(set(caliper["Well"])) - i - 1))
    visibility.append(vis)

    # Add button for this well
    buttons.append(dict(
        label=well,
        method="update",
        args=[{"visible": vis},
              {"title": f"{well} – Caliper and Conductivity (Scatter View)"}]
    ))

# Add interactive menu to the figure
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        buttons=buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title="Caliper and Conductivity Logs",
    height=600,
    yaxis=dict(title="Depth (m)", autorange="reversed"),
    showlegend=False,
    margin=dict(t=100, l=60, r=60, b=60)
)

# Set axis titles
fig.update_xaxes(title_text="Conductivity (µS/cm)", row=1, col=1)
fig.update_xaxes(title_text="Caliper", row=1, col=2)

# Export to HTML
fig.write_html("caliper_conductivity_logs_interactive.html")
